In [ ]:
import os
os.chdir("/content/drive/MyDrive/Colab Notebooks/Lab2")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime

In [ ]:
# Patikrinama, ar yra GPU
print(torch.cuda.is_available())
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
print(device)

In [ ]:
# Transformacijos
transforms_train = transforms.Compose([
  transforms.Resize((128, 128)),
  transforms.ToTensor(),
  transforms.RandomRotation(15),
  transforms.RandomGrayscale(),
  transforms.RandomHorizontalFlip(),
  transforms.ColorJitter(0.2, 0.3, 0.2),
  transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
])

transforms_test = transforms.Compose([
  transforms.Resize((128, 128)),
  transforms.ToTensor(),
  transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
])

In [ ]:
batch_size = 16
num_workers = 2

# Užkrauna mokymosi duomenis
trainset = torchvision.datasets.ImageFolder(root='/content/drive/MyDrive/Colab Notebooks/Lab2/OIDv4_ToolKit/OID/Dataset/train', transform=transforms_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, num_workers=num_workers, shuffle=True)

# Užkrauna testavimo duomenis
testset = torchvision.datasets.ImageFolder(root='/content/drive/MyDrive/Colab Notebooks/Lab2/OIDv4_ToolKit/OID/Dataset/test', transform=transforms_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, num_workers=num_workers, shuffle=False)

classes = ('dog', 'snake', 'fish')
print(f'Train: {len(trainset)}, Test: {len(testset)}')

In [ ]:
# Čia sukurti du konvoliuciniai sluoksniai (conv1 ir conv2) su atitinkamais baseino sluoksniais
# (pool1 ir pool2). Conv2d sluoksniai atsakingi už vaizdo savybių išgavimą, o MaxPool2d ir AvgPool2d
# sluoksniai sumažina išvesties dimensijas (aukštį ir plotį), taip sumažinant apdorojamų duomenų kiekį
# ir išgautų savybių abstrakcijos lygį.
class ConvNet(torch.nn.Module):
  def __init__(self, in_shape, out_classes):
    super().__init__()
    self.conv1 = torch.nn.Conv2d(in_shape[0], 16, (3, 3), padding = 'same')
    self.pool1 = torch.nn.MaxPool2d((2, 2), (2, 2))
    self.conv2 = torch.nn.Conv2d(16, 16, (3, 3), padding = 'same')
    self.pool2 = torch.nn.AvgPool2d((2, 2), (2, 2))
    # Apskaičiuoja, kiek neuronų reikės šiame sluoksnyje, atsižvelgiant į pradinę įvestį i
    # ir tai, kad du sluoksniai sumažina išmatavimus 4 kartus.
    self.fc = torch.nn.Linear(16 * in_shape[1] * in_shape[2] // 4**2, out_classes)

  # apibrėžia, kaip modelis atlieks prognozes
  def forward(self, x):
    y = torch.nn.Sequential(
      self.conv1,
      torch.nn.ReLU(),
      self.pool1,
      self.conv2,
      torch.nn.ReLU(),
      self.pool2,
      torch.nn.Flatten(),
      self.fc
    )(x)
    return y

In [ ]:
# Gradientas naudojamas norint nustatyti, kaip reikėtų keisti
# modelio parametrus (svorius), siekiant mažinti praradimo funkcijos reikšmę

#  Kiekvienas ryšys tarp neuronų turi priskirtą svorį, kuris nurodo, kaip jie veikia vienas kitą.
def train(model, loader, classes, epoch_count, lr=1e-3):
  loss_func = torch.nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(model.parameters(), lr=lr)
  model.train()

  start_time = datetime.now()

  for epoch in range(epoch_count):
    loss_values = []
    correct = 0
    total = 0

    for data in loader:
      images, labels = data
      images, labels = images.to(device), labels.to(device)

      # Pakeičia kiekvieno svorio reikšmę taip, kad praradimo funkcijos reikšmė būtų mažesnė
      optimizer.zero_grad()
      pred = model(images)
      loss = loss_func(pred, labels)
      loss_values.append(loss.item())
      # Skaičiuojami praradimo funkcijos gradientai pagal visus modelio svorius
      # Nustato, kaip pasikeis praradimo funkcijos reikšmė, keičiant kiekvieną modelio svorį.
      # Tai reiškia, kad loss.backward() automatiškai apskaičiuoja, kaip turėtų būti pakeisti modelio svoriai,
      # siekiant sumažinti praradimo funkcijos reikšmę, ir įrašo šiuos gradientus į atitinkamus svorių objektus.
      loss.backward()
      optimizer.step()

      _, predicted = torch.max(pred.data, 1)
      total += labels.size(0)
      correct += (predicted == labels).sum().item()

    epoch_loss = np.mean(loss_values)
    epoch_accuracy = correct / total * 100
    current_time = datetime.now()
    elapsed_time = (current_time - start_time).total_seconds()

    print(f'Epoch {epoch + 1}/{epoch_count}, Loss: {epoch_loss:.4f}, '
          f'Accuracy: {epoch_accuracy:.2f}%, Time: {elapsed_time:.2f}s')

In [ ]:
class_count = len(trainset.classes)
model = ConvNet(trainset[0][0].shape, class_count).to(device)

# Užkrauna svorius iš buvusio mokymosi ciklo
saved_model_path = '/content/drive/MyDrive/Colab Notebooks/Lab2/saved_model.pth'
if os.path.exists(saved_model_path):
  model.load_state_dict(torch.load(saved_model_path, map_location=torch.device(device)))

train(model, trainloader, class_count, epoch_count = 30)
torch.save(model.state_dict(), saved_model_path)

In [109]:
# Modelio testavimas
model.load_state_dict(torch.load(saved_model_path))
model.eval()

all_preds = []
all_labels = []

with torch.no_grad():
  for images, labels in testloader:
    images, labels = images.to(device), labels.to(device)

    # Gauna prognozes
    outputs = model(images)
    _, predicted = torch.max(outputs, 1)

    # Prognozuotos ir teisingos žymės
    all_preds.extend(predicted.cpu().numpy())
    all_labels.extend(labels.cpu().numpy())

In [ ]:
accuracy = accuracy_score(all_labels, all_preds)
precision = precision_score(all_labels, all_preds, average='weighted')
recall = recall_score(all_labels, all_preds, average='weighted')
f1 = f1_score(all_labels, all_preds, average='weighted')
conf_matrix = confusion_matrix(all_labels, all_preds)

print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')

sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=classes, yticklabels=classes)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.show()